### Necessary Libraries

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 4.9 MB 16.3 MB/s 
     |████████████████████████████████| 1.3 MB 48.1 MB/s 
     |████████████████████████████████| 163 kB 48.4 MB/s 
     |████████████████████████████████| 6.6 MB 40.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=200e1071dc488a4d2d57b17e2b0a94c2846669ff25d29246ea921070f8914726
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import model_from_json
from keras.models import load_model
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn import linear_model, metrics

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32165 entries, 0 to 32164
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   patient_id                  32165 non-null  int64  
 1   name_of_drug                32165 non-null  object 
 2   use_case_for_drug           32165 non-null  object 
 3   review_by_patient           32165 non-null  object 
 4   effectiveness_rating        32165 non-null  int64  
 5   drug_approved_by_UIC        32165 non-null  object 
 6   number_of_times_prescribed  32165 non-null  int64  
 7   base_score                  32165 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 2.0+ MB


In [ ]:
for i in range(10):
  print(df.review_by_patient[i])

"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"
"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. 
We have tried many different medications and so far this is the most effective."
"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene,

In [ ]:
sample = '27'
#df.review_by_patient[0]
sentences = [sample]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
# print(embeddings)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,patient_id,effectiveness_rating,number_of_times_prescribed,base_score
patient_id,1.000000,0.018803,0.017499,0.010924
effectiveness_rating,0.018803,1.000000,0.242224,0.412301
number_of_times_prescribed,0.017499,0.242224,1.000000,0.134295
base_score,0.010924,0.412301,0.134295,1.000000


In [ ]:
def get_embeddings(x):
  return model.encode(x)

In [ ]:
df_emd = df.copy()

### Preprocessing our data

In [ ]:
df_emd['effectiveness_rating'] = df_emd['effectiveness_rating'].astype(str)
df_emd['drug_approved_by_UIC'] = df_emd['drug_approved_by_UIC'].astype(str)
df_emd['number_of_times_prescribed'] = df_emd['number_of_times_prescribed'].astype(str)

In [ ]:
df_emd.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score'],
      dtype='object')

In [ ]:
df_emd

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782
...,...,...,...,...,...,...,...,...
32160,183202,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",9,10-Jun-13,89,6.963020
32161,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,0.899076
32162,121154,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",9,10-Nov-16,25,6.241812
32163,45410,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",8,21-Jan-15,22,7.940428


In [ ]:
df_emd['use_case_for_drug'] = df_emd['use_case_for_drug'].apply(get_embeddings)

In [ ]:
df_emd['review_by_patient'] = df_emd['review_by_patient'].apply(get_embeddings)

In [ ]:
df_emd['effectiveness_rating'] = df_emd['effectiveness_rating'].apply(get_embeddings)

In [ ]:
df_emd['number_of_times_prescribed'] = df_emd['number_of_times_prescribed'].apply(get_embeddings)

In [ ]:
df_emd

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,"[0.04402538, 0.023435835, 0.012823722, 0.11487...","[-0.016037412, 0.022653574, 0.0031208405, 0.04...","[-0.005709678, 0.010430608, -0.048066407, -0.0...",20-May-12,"[-0.03572919, 0.03401141, -0.08050072, 0.04785...",8.022969
1,95260,Guanfacine,"[0.037006058, 0.005946991, -0.013087106, 0.089...","[0.019416332, 0.049999256, -0.05314657, 0.0219...","[-0.017075757, 0.048522595, -0.056910973, 0.07...",27-Apr-10,"[-0.016878331, -0.0018649985, -0.07022963, -0....",7.858458
2,92703,Lybrel,"[-0.09198214, 0.037292663, -0.056692082, 0.049...","[-0.016496643, -0.06817713, -0.04971898, 0.039...","[-0.041312058, -0.006703298, -0.031351108, 0.0...",14-Dec-09,"[0.0017547084, 0.08040906, -0.055555653, -0.00...",6.341969
3,35696,Buprenorphine / naloxone,"[0.027867591, -0.037415624, -0.050747894, 0.10...","[-0.03744404, -0.023789791, 0.028102916, 0.118...","[-0.005709678, 0.010430608, -0.048066407, -0.0...",27-Nov-16,"[0.028754188, 0.068190165, -0.09859019, 0.0104...",6.590176
4,155963,Cialis,"[0.0013151935, 0.05938213, 0.0004070892, 0.014...","[-0.045738272, -0.0240489, 0.098223135, 0.0732...","[-0.028164776, -0.024786498, -0.09065127, 0.04...",28-Nov-15,"[0.08462971, 0.07418376, -0.053185947, 0.02646...",6.144782
...,...,...,...,...,...,...,...,...
32160,183202,Cymbalta,"[0.052281413, 0.029752295, 0.0038559793, 0.130...","[0.013371257, 0.036854327, 0.013577041, 0.0192...","[-0.005709678, 0.010430608, -0.048066407, -0.0...",10-Jun-13,"[-0.0027857034, 0.11871025, -0.04058412, 0.061...",6.963020
32161,109111,Nexplanon,"[-0.09198214, 0.037292663, -0.056692082, 0.049...","[-0.093116, -0.018769315, 0.044224333, 0.04980...","[-0.003321721, 0.004443901, -0.08935944, 0.042...",6-Apr-17,"[0.039207794, 0.05865119, -0.09762955, 0.08444...",0.899076
32162,121154,Venlafaxine,"[-0.003509115, -0.006937782, 0.020086678, -0.0...","[0.023151431, 0.02236962, -0.0189768, 0.083352...","[-0.005709678, 0.010430608, -0.048066407, -0.0...",10-Nov-16,"[-0.00060694694, 0.038072083, -0.07132152, 0.0...",6.241812
32163,45410,Fluoxetine,"[0.090759344, -0.042512514, 0.0039732037, 0.03...","[0.0066768145, 0.03221691, -0.0401424, 0.12254...","[-0.017075757, 0.048522595, -0.056910973, 0.07...",21-Jan-15,"[0.03933437, 0.11609399, -0.046480555, 0.05504...",7.940428


### Get Train and test df 

In [ ]:
X = df_emd[['use_case_for_drug','review_by_patient','effectiveness_rating','number_of_times_prescribed']].to_numpy()
y = df_emd[['base_score']]

In [ ]:
X = np.array([np.reshape(x, (1, len(X[0][0]))) 
              for i in range(len(X)) 
              for x in X[i]]).reshape(-1, 4*len(X[0][0])).astype(np.float32)

print(X.shape)
print(X[0].shape)

(32165, 1536)
(1536,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(21550, 1536)
(10615, 1536)
(21550, 1)
(10615, 1)


### Define Loss

In [ ]:
msle = MeanSquaredLogarithmicError()

### Define Model and Train

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(160, kernel_initializer='normal', activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(480, kernel_initializer='normal', activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, kernel_initializer='normal', activation='relu'),
    keras.layers.Dense(1, kernel_initializer='normal', activation='linear')
  ])

model.compile(
    loss=msle, 
    optimizer=keras.optimizers.Adam(learning_rate=0.01), 
    metrics=[msle]
)
# train the model
history = model.fit(
    X_train, 
    y_train, 
    epochs=30, 
    batch_size=64,
    validation_split=0.2
)

Epoch 1/30
270/270 [==============================] - 4s 10ms/step - loss: 0.0533 - mean_squared_logarithmic_error: 0.0532 - val_loss: 0.0118 - val_mean_squared_logarithmic_error: 0.0120
Epoch 2/30
270/270 [==============================] - 3s 10ms/step - loss: 0.0151 - mean_squared_logarithmic_error: 0.0152 - val_loss: 0.0087 - val_mean_squared_logarithmic_error: 0.0089
Epoch 3/30
270/270 [==============================] - 3s 10ms/step - loss: 0.0136 - mean_squared_logarithmic_error: 0.0136 - val_loss: 0.0070 - val_mean_squared_logarithmic_error: 0.0070
Epoch 4/30
270/270 [==============================] - 3s 9ms/step - loss: 0.0115 - mean_squared_logarithmic_error: 0.0114 - val_loss: 0.0083 - val_mean_squared_logarithmic_error: 0.0084
Epoch 5/30
270/270 [==============================] - 2s 9ms/step - loss: 0.0095 - mean_squared_logarithmic_error: 0.0095 - val_loss: 0.0090 - val_mean_squared_logarithmic_error: 0.0091
Epoch 6/30
270/270 [==============================] - 3s 9ms/step -

In [ ]:
# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_num.h5")

In [ ]:
# load json and create model
json_file = open('model_num.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_num.h5")
print("Loaded model from disk")

loaded_model.save('model_num.hdf5')
loaded_model=load_model('model_num.hdf5')

Loaded model from disk


In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)
  
# regression coefficients
print('Coefficients: ', reg.coef_)
result = reg.score(X_test, y_test)
print(result)

Coefficients:  [[-0.06851184 -0.24300364 -1.1203021  ... 31.50445    17.70889
  19.68141   ]]
0.957459253167528


## Testing

In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
class ScoreModel():
  def __init__(self):
    self.test_df = pd.read_csv('test.csv')
    self.loaded_model = load_model('model_num.hdf5')
    self.embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    self.df = pd.read_csv('train.csv')
    print('Model Initialized')
  
  def get_score(self, patientID):
    '''
    When patientID is from test set use get_score.
    '''
    a = self.test_df.loc[test_df['patient_id']==patientID].values
    f1 = a[0][5]
    f2 = a[0][2]
    f3 = a[0][6]
    f4 = a[0][4]
    self.emd_f1 = self.embedder.encode(str(f1))
    self.emd_f2 = self.embedder.encode(str(f2))
    self.emd_f3 = self.embedder.encode(str(f3))
    self.emd_f4 = self.embedder.encode(str(f4))
    sample = np.array([self.emd_f1, self.emd_f2, self.emd_f3, self.emd_f4])
    out = self.loaded_model.predict(sample.reshape(1, -1))
    return out[0][0]

  def check_score(self, patientID):
    '''
    When patientID is from train set use check_score.
    '''
    a = self.df.loc[df['patient_id']==patientID].values
    f1 = a[0][5]
    f2 = a[0][2]
    f3 = a[0][6]
    f4 = a[0][4]
    self.emd_f1 = self.embedder.encode(str(f1))
    self.emd_f2 = self.embedder.encode(str(f2))
    self.emd_f3 = self.embedder.encode(str(f3))
    self.emd_f4 = self.embedder.encode(str(f4))
    sample = np.array([self.emd_f1, self.emd_f2, self.emd_f3, self.emd_f4])
    out = self.loaded_model.predict(sample.reshape(1, -1))
    return out[0][0]

In [ ]:
test_model = ScoreModel()

Model Initialized


### Test sample from train.csv

In [ ]:
df.loc[13230]

patient_id                                                               153544
name_of_drug                                                        Doxycycline
use_case_for_drug                                                          Acne
review_by_patient             "I have had problems with acne all my life. I ...
effectiveness_rating                                                         10
drug_approved_by_UIC                                                  10-Mar-12
number_of_times_prescribed                                                   38
base_score                                                             6.181254
Name: 13230, dtype: object

In [ ]:
test_model.check_score(153544)

6.1940374

In [ ]:
df.loc[12345]

patient_id                                                                46867
name_of_drug                                                            Xenical
use_case_for_drug                                                       Obesity
review_by_patient             "I love Xenical. It is basically making me sca...
effectiveness_rating                                                         10
drug_approved_by_UIC                                                  26-Nov-11
number_of_times_prescribed                                                  250
base_score                                                             7.229325
Name: 12345, dtype: object

In [ ]:
test_model.check_score(46867)

6.255546

In [ ]:
df.loc[19999]

patient_id                                                               221074
name_of_drug                                                             Drysol
use_case_for_drug                                                 Hyperhidrosis
review_by_patient             "I&#039;m am a 18 year old girl whose been swe...
effectiveness_rating                                                          9
drug_approved_by_UIC                                                  25-Aug-17
number_of_times_prescribed                                                   10
base_score                                                              8.67158
Name: 19999, dtype: object

In [ ]:
test_model.check_score(221074)

7.3930798

In [ ]:
df.loc[14599]

patient_id                                                               199094
name_of_drug                                                       Depo-Provera
use_case_for_drug                                     Abnormal Uterine Bleeding
review_by_patient             "I got the shot for the first time in December...
effectiveness_rating                                                          1
drug_approved_by_UIC                                                   9-Mar-16
number_of_times_prescribed                                                    5
base_score                                                             4.884977
Name: 14599, dtype: object

In [ ]:
print(type(199094))
test_model.check_score(199094)

<class 'int'>


4.0392385

### test Samples from test.csv

In [ ]:
result = []
for i in range(10760):
  a = int(test_df['patient_id'][i])
  x = test_model.get_score(a)
  result.append(x)

In [ ]:
test_df['result'] = result

In [ ]:
test_df

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating,result
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10,7.741059
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9,7.493814
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4,5.350338
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7,5.387123
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2,5.227960
...,...,...,...,...,...,...,...,...
10755,151266,Chantix,"""I took chantix a little over a month. It made...",11-Nov-17,2,Smoking Cessation,1,5.607785
10756,139347,Armodafinil,"""This medicine kept me from sleeping the whole...",30-Sep-14,18,Narcolepsy,1,4.367601
10757,159999,Tamoxifen,"""I have taken Tamoxifen for 5 years. Side effe...",13-Sep-14,43,"Breast Cancer, Prevention",10,6.117405
10758,130945,Levonorgestrel,"""I&#039;m married, 34 years old and I have no ...",15-Nov-10,7,Birth Control,8,4.923341


In [ ]:
test_df.to_csv('submission.csv')